In [5]:
# =========================
# SETUP
# =========================

# Install dependencies
!pip install gluonts pandas numpy datasets pyarrow -q

# Import common libraries
import pandas as pd
import numpy as np
import os

# Create folder to save outputs
os.makedirs("datasets_csv", exist_ok=True)

print("Setup done ✅")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==

In [7]:
# =========================
# M4 Monthly
# =========================

# Import
from gluonts.dataset.repository.datasets import get_dataset

# Download
m4_dataset = get_dataset("m4_monthly")

# Extract and process
train_list = []
for i, entry in enumerate(m4_dataset.train):
    if i >= 100:  # Only 100 samples
        break
    ts = entry['target']
    id_ = entry['item_id']
    df = pd.DataFrame({
        'id': [id_] * len(ts),
        'value': ts,
        'timestamp': pd.date_range(start=entry['start'].to_timestamp(), periods=len(ts), freq='M')
    })
    train_list.append(df)

# Combine
m4_monthly_df = pd.concat(train_list, ignore_index=True)

# Save
m4_monthly_df.to_csv("datasets_csv/m4_monthly.csv", index=False)
print("Saved M4 monthly dataset ✅")


<ipython-input-7-a48b07437489>:21: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'timestamp': pd.date_range(start=entry['start'].to_timestamp(), periods=len(ts), freq='M')


Saved M4 monthly dataset ✅


In [9]:
# =========================
# GluonTS Datasets
# =========================

# Function to download and save GluonTS dataset
def save_gluonts_dataset(name, filename_prefix):
    from gluonts.dataset.repository.datasets import get_dataset

    dataset = get_dataset(name)
    dfs = []
    count = 0
    for entry in dataset.train:
        if count >= 100:  # 100 samples only
            break
        ts = entry['target']
        id_ = entry['item_id']
        df = pd.DataFrame({
            'id': [id_] * len(ts),
            'value': ts,
            'timestamp': pd.date_range(start=entry['start'].to_timestamp(), periods=len(ts), freq='H')
        })
        dfs.append(df)
        count += 1
    final_df = pd.concat(dfs, ignore_index=True)
    final_df.to_csv(f"datasets_csv/{filename_prefix}.csv", index=False)
    print(f"Saved {filename_prefix} dataset ✅")

# Download and Save
save_gluonts_dataset("electricity", "gluonts_electricity")
save_gluonts_dataset("traffic", "gluonts_traffic")
save_gluonts_dataset("exchange_rate", "gluonts_exchange_rate")


<ipython-input-9-07b817fb14e2>:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'timestamp': pd.date_range(start=entry['start'].to_timestamp(), periods=len(ts), freq='H')


Saved gluonts_electricity dataset ✅


<ipython-input-9-07b817fb14e2>:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'timestamp': pd.date_range(start=entry['start'].to_timestamp(), periods=len(ts), freq='H')


Saved gluonts_traffic dataset ✅


/usr/local/lib/python3.11/dist-packages/gluonts/dataset/repository/_lstnet.py:133: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  time_index = pd.period_range(
/usr/local/lib/python3.11/dist-packages/gluonts/dataset/repository/_lstnet.py:133: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  time_index = pd.period_range(
/usr/local/lib/python3.11/dist-packages/gluonts/dataset/repository/_lstnet.py:177: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  training_end + i * ds_info.prediction_length
/usr/local/lib/python3.11/dist-packages/gluonts/dataset/repository/_lstnet.py:186: FutureWarning: Period with BDay freq is deprecated and will be removed in a future version. Use a DatetimeIndex with BDay freq instead.
  prediction_start_dat

Saved gluonts_exchange_rate dataset ✅


In [10]:
# =========================
# ETTm2 Dataset
# =========================

# Import
import pandas as pd

# Download the ETTm2 dataset
!wget -q https://raw.githubusercontent.com/zhouhaoyi/ETDataset/main/ETT-small/ETTm2.csv -P datasets_csv/

# Load
ett_df = pd.read_csv("datasets_csv/ETTm2.csv")
ett_df['date'] = pd.to_datetime(ett_df['date'])

# Resample to Monthly mean
ett_monthly = ett_df.set_index('date').resample('M').mean().reset_index()

# Take first 100 timestamps for simplicity
ett_monthly_sampled = ett_monthly.head(100)

# Save
ett_monthly_sampled.to_csv("datasets_csv/ettm2_monthly.csv", index=False)
print("Saved ETTm2 monthly dataset ✅")


Saved ETTm2 monthly dataset ✅


<ipython-input-10-eaf1aafb0fa6>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ett_monthly = ett_df.set_index('date').resample('M').mean().reset_index()


In [11]:
# =========================
# BIGDATA22 (Simulated)
# =========================

# Import
import numpy as np
import pandas as pd

# Simulate stock price data
dates = pd.date_range(start='2010-01-01', end='2020-12-31', freq='B')  # Business days
np.random.seed(42)

stocks = [f'Stock{i}' for i in range(5)]  # Simulate 5 stock series
data = {stock: 100 + np.cumsum(np.random.randn(len(dates))) for stock in stocks}
bigdata_df = pd.DataFrame(data, index=dates)

# Resample to Monthly
bigdata_monthly = bigdata_df.resample('M').last().reset_index()

# Melt into long format
bigdata_monthly_melted = bigdata_monthly.melt(id_vars=['index'], var_name='id', value_name='value')
bigdata_monthly_melted.rename(columns={'index': 'timestamp'}, inplace=True)

# Take 100 rows
bigdata_monthly_sampled = bigdata_monthly_melted.head(100)

# Save
bigdata_monthly_sampled.to_csv("datasets_csv/bigdata22_monthly.csv", index=False)
print("Saved BIGDATA22 monthly dataset ✅")


Saved BIGDATA22 monthly dataset ✅


<ipython-input-11-58df6dcd0006>:18: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  bigdata_monthly = bigdata_df.resample('M').last().reset_index()


In [13]:
# =========================
# Weather Dataset
# =========================

# Import
import pandas as pd

# Download sample weather data
!wget -q https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv -P datasets_csv/

# Load
weather_df = pd.read_csv("datasets_csv/daily-min-temperatures.csv")
weather_df['Date'] = pd.to_datetime(weather_df['Date'])

# Resample to Monthly mean
weather_monthly = weather_df.set_index('Date').resample('M').mean().reset_index()

# Take first 100 rows
weather_monthly_sampled = weather_monthly.head(100)

# Save
weather_monthly_sampled.to_csv("datasets_csv/weather_monthly.csv", index=False)
print("Saved Weather monthly dataset ✅")


Saved Weather monthly dataset ✅


<ipython-input-13-98e8a86b0571>:16: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  weather_monthly = weather_df.set_index('Date').resample('M').mean().reset_index()


In [14]:
# First zip the folder
!zip -r datasets_csv.zip datasets_csv

# Then download
from google.colab import files
files.download('datasets_csv.zip')


  adding: datasets_csv/ (stored 0%)
  adding: datasets_csv/m4_monthly.csv (deflated 76%)
  adding: datasets_csv/daily-min-temperatures.csv (deflated 79%)
  adding: datasets_csv/ETTm2.csv (deflated 84%)
  adding: datasets_csv/gluonts_electricity.csv (deflated 86%)
  adding: datasets_csv/ettm2_monthly.csv (deflated 48%)
  adding: datasets_csv/weather_monthly.csv (deflated 74%)
  adding: datasets_csv/gluonts_traffic.csv (deflated 85%)
  adding: datasets_csv/bigdata22_monthly.csv (deflated 64%)
  adding: datasets_csv/gluonts_exchange_rate.csv (deflated 83%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>